<a href="https://colab.research.google.com/github/cebisiles/unsupervised-predict-streamlit-template/blob/master/Group_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Import libraries
import numpy as np
import pandas as pd

# Reading ratings file
ratings = pd.read_csv('https://raw.githubusercontent.com/cebisiles/unsupervised-predict-streamlit-template/master/resources/data/ratings.csv')

# Reading users file
#users = pd.read_csv('users.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])

# Reading movies file
movies = pd.read_csv('https://raw.githubusercontent.com/cebisiles/unsupervised-predict-streamlit-template/master/resources/data/movies.csv')

In [2]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
n_users = ratings.userId.unique().shape[0]
n_movies = ratings.movieId.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 671 | Number of movies = 9066


In [5]:
ratings = ratings.sample(10000)

In [6]:
Ratings = ratings.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
Ratings.head()

movieId,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,28,29,30,31,32,34,35,36,39,40,41,43,44,45,47,48,...,126548,127152,127198,127319,128512,130960,130980,132888,134130,134368,134783,134853,135268,135887,136020,136305,136602,137595,138036,139644,140711,140816,141956,143377,143385,144976,145839,146443,147006,147426,148372,148626,149352,149532,150401,151639,152077,152081,152173,160438
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
R = Ratings.to_numpy()
user_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [8]:
sparsity = round(1.0 - len(ratings) / float(n_users * n_movies), 3)
print('The sparsity level of Movie dataset is ' +  str(sparsity * 100) + '%')

The sparsity level of Movie dataset is 99.8%


In [9]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(Ratings_demeaned, k = 50)

In [10]:
sigma = np.diag(sigma)

In [11]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [12]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = Ratings.columns)
preds.head()

movieId,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,28,29,30,31,32,34,35,36,39,40,41,43,44,45,47,48,...,126548,127152,127198,127319,128512,130960,130980,132888,134130,134368,134783,134853,135268,135887,136020,136305,136602,137595,138036,139644,140711,140816,141956,143377,143385,144976,145839,146443,147006,147426,148372,148626,149352,149532,150401,151639,152077,152081,152173,160438
0,-0.095837,0.009523,-0.009002,0.006240,-0.047417,0.035983,0.031220,0.002330,-0.005033,0.000247,-0.000764,0.004507,0.002085,0.000969,0.020546,0.094324,-0.004222,0.008346,0.001928,0.019210,-0.008045,0.002294,0.010777,-0.017918,0.002926,0.019477,0.002603,-0.000926,-0.044248,-0.012195,0.001947,0.053306,-0.007388,-0.004933,0.026337,-0.004014,0.016865,0.001238,0.071115,0.015886,...,-0.000041,0.029597,-0.001847,-0.001330,-0.002290,-0.014377,0.001159,-0.003845,-0.001409,-0.003261,-0.000233,-0.002005,-0.016466,-0.001872,-0.004173,-0.000805,-0.000418,-0.000233,-0.005270,-0.005241,-0.003806,-0.003845,-0.000086,-0.003845,-0.003806,-0.005050,-0.001374,-0.012288,-0.001066,-0.001285,0.002165,0.001864,-0.004617,-0.000358,-0.002907,-0.000584,-0.004422,-0.002067,-0.000750,-0.001555
1,0.035387,-0.001143,0.032560,0.006462,0.033680,0.002328,-0.007000,-0.004684,-0.055189,0.004409,0.006023,0.010125,0.012892,0.002058,-0.042804,-0.024417,-0.004366,0.015822,-0.002621,0.079634,-0.007413,0.004453,0.027285,-0.005609,0.031730,0.061029,0.000125,-0.001221,0.186437,0.209001,0.007599,0.002777,0.065011,0.009795,0.014003,-0.003198,0.064653,0.029514,-0.053824,0.010745,...,-0.000806,-0.007701,-0.009748,-0.002794,-0.007264,-0.009252,-0.009360,-0.005605,-0.010188,-0.004923,-0.001415,-0.005448,-0.010455,-0.000233,-0.006974,-0.002141,-0.003000,-0.001415,0.008880,-0.003783,-0.002364,-0.005605,-0.001918,-0.005605,-0.002364,0.000669,-0.001446,-0.008050,0.002872,0.003490,0.003363,-0.010480,-0.002670,-0.010954,-0.004754,-0.003542,0.029185,-0.007415,-0.004083,-0.004525
2,-0.035733,-0.027796,-0.056808,-0.025896,-0.038700,-0.115365,-0.060789,-0.019845,-0.093017,-0.021530,-0.021873,0.048473,-0.004441,-0.003034,0.297699,-0.046370,-0.020008,-0.062465,-0.011710,-0.136028,-0.028374,-0.018448,-0.017877,0.084229,0.037838,-0.048461,-0.012284,-0.005988,-0.127019,0.233533,-0.007875,-0.024205,-0.095289,-0.022800,-0.019018,0.008481,-0.008291,0.033369,0.147518,0.026843,...,-0.010838,0.053280,0.026117,0.000307,-0.012796,0.021774,-0.059110,-0.020178,0.068180,-0.018156,-0.006514,0.036241,0.025745,-0.033469,-0.013589,-0.001803,0.036994,-0.006514,-0.010439,-0.030933,0.036252,-0.020178,0.015486,-0.020178,0.036252,-0.074640,0.010888,0.017804,-0.007397,-0.007626,-0.016516,0.010651,0.044707,0.009283,0.006634,0.047748,-0.021600,-0.013089,0.058502,0.027788
3,-0.533039,0.282771,0.298022,-0.014129,-0.331670,0.139191,0.136011,0.006888,0.634830,-0.065271,-0.023759,0.041841,-0.029761,0.015465,-0.188541,0.983054,-0.006682,0.282460,0.013306,0.030325,0.124928,0.018178,-0.063292,-0.010020,-0.015500,0.669780,-0.038394,-0.047582,-0.234919,-0.508196,-0.031671,0.172469,0.152174,0.049087,0.017251,-0.041001,0.070461,0.016799,1.145468,-0.120013,...,-0.016362,-0.042383,0.087360,0.013559,-0.036079,-0.325986,0.052878,-0.028419,0.047803,-0.025565,-0.013629,-0.016016,-0.371349,-0.031536,0.055703,0.006225,0.033425,-0.013629,-0.052939,-0.099262,-0.005916,-0.028419,0.012492,-0.028419,-0.005916,-0.066177,-0.007431,-0.280622,-0.040375,-0.045697,0.131456,-0.107517,-0.005412,-0.038708,0.035558,0.043892,0.015129,-0.033698,0.054358,-0.014501
4,0.012576,-0.016245,0.096059,-0.004952,-0.014822,0.051710,0.037669,0.022885,0.128479,-0.014149,0.002897,-0.020456,-0.015184,-0.002156,0.058385,0.081233,0.004875,0.044447,0.000215,0.044593,0.058841,0.002137,-0.015923,-0.066609,0.008022,-0.025949,-0.005754,-0.000241,0.064187,-0.070054,0.002969,-0.000363,0.015123,0.016950,0.016422,-0.009473,-0.024890,0.006721,0.134969,0.029489,...,0.004253,0.043630,0.009436,0.035617,0.021823,-0.071684,-0.009579,0.004013,0.006947,0.003496,0.004820,-0.006634,-0.081

In [13]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.userId == (userID)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movieId', right_on = 'movieId').sort_values(['rating'], ascending=False))

    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['movieId'].isin(user_full['movieId'])].merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',left_on = 'movieId',right_on = 'movieId').rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :-1])

    return user_full, recommendations

In [14]:
already_rated, predictions = recommend_movies(preds, 89, movies, ratings, 20)

User 89 has already rated 7 movies.
Recommending highest 20 predicted ratings movies not already rated.


In [15]:
already_rated.head(20)

,userId,movieId,rating,timestamp,title,genres
0,89,8528,5.0,1257610580,Dodgeball: A True Underdog Story (2004),Comedy
1,89,2959,5.0,1257620271,Fight Club (1999),Action|Crime|Drama|Thriller
3,89,2762,5.0,1257620518,"Sixth Sense, The (1999)",Drama|Horror|Mystery
4,89,590,5.0,1257620132,Dances with Wolves (1990),Adventure|Drama|Western
5,89,3793,5.0,1257620282,X-Men (2000),Action|Adventure|Sci-Fi
6,89,500,5.0,1257620515,Mrs. Doubtfire (1993),Comedy|Drama
2,89,2987,4.0,1257620535,Who Framed Roger Rabbit? (1988),Adventure|Animation|Children|Comedy|Crime|Fant...


In [16]:
predictions

,movieId,title,genres
351,356,Forrest Gump (1994),Comedy|Drama|Romance|War
33,34,Babe (1995),Children|Drama
2606,2700,"South Park: Bigger, Longer and Uncut (1999)",Animation|Comedy|Musical
2478,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
535,541,Blade Runner (1982),Action|Sci-Fi|Thriller
7022,7153,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy
108,110,Braveheart (1995),Action|Drama|War
289,293,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller
583,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
582,592,Batman (1989),Action|Crime|Thriller


In [18]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 6.0 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1623210 sha256=dd1349c79a55046a043617f8ba9abea185e65df677259743e3a95180da83712e
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [19]:
# Import libraries from Surprise package
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

# Load Reader library
reader = Reader()

# Load ratings dataset with Dataset library
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [20]:
# Use the SVD algorithm.
svd = SVD()

# Compute the RMSE of the SVD algorithm.
cross_validate(svd, data, measures=['RMSE'], cv = 5)

{'fit_time': (1.3558433055877686,
  1.0854613780975342,
  1.6819124221801758,
  1.428421974182129,
  0.9756920337677002),
 'test_rmse': array([0.97114759, 0.99613855, 0.96687516, 0.94485929, 0.95731153]),
 'test_time': (0.10274791717529297,
  0.05684924125671387,
  0.08222699165344238,
  0.023940086364746094,
  0.014724016189575195)}

In [21]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [22]:
ratings[ratings['userId'] == 9041]

,userId,movieId,rating,timestamp


In [27]:
predictions = lambda x: svd.predict(x['userId'], x['movieId']).est

In [29]:
print(predictions)

<function <lambda> at 0x7f12542529e0>


In [24]:
# Import Keras libraries
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
# Import CF Model Architecture
from CFModel import CFModel

ModuleNotFoundError: ignored

In [ ]:
# Define model
model = CFModel(max_userid, max_movieid, K_FACTORS)
# Compile the model using MSE as the loss function and the AdaMax learning algorithm
model.compile(loss='mse', optimizer='adamax')

In [ ]:
# Callbacks monitor the validation loss
# Save the model weights each time the validation loss has improved
callbacks = [EarlyStopping('val_loss', patience=2), 
             ModelCheckpoint('weights.h5', save_best_only=True)]

# Use 30 epochs, 90% training data, 10% validation data 
history = model.fit([Users, Movies], Ratings, nb_epoch=30, validation_split=.1, verbose=2, callbacks=callbacks)

In [ ]:
# Show the best validation RMSE
min_val_loss, idx = min((val, idx) for (idx, val) in enumerate(history.history['val_loss']))
print 'Minimum RMSE at epoch', '{:d}'.format(idx+1), '=', '{:.4f}'.format(math.sqrt(min_val_loss))

In [ ]:
# Use the pre-trained model
trained_model = CFModel(max_userid, max_movieid, K_FACTORS)
# Load weights
trained_model.load_weights('weights.h5')

In [ ]:
# Function to predict the ratings given User ID and Movie ID
def predict_rating(user_id, movie_id):
    return trained_model.rate(user_id - 1, movie_id - 1)

In [ ]:
# Define constants
K_FACTORS = 100 # The number of dimensional embeddings for movies and users
TEST_USER = 2000 # A random test user (user_id = 2000)

In [ ]:
user_ratings = ratings[ratings['user_id'] == TEST_USER][['user_id', 'movie_id', 'rating']]
user_ratings['prediction'] = user_ratings.apply(lambda x: predict_rating(TEST_USER, x['movie_id']), axis=1)
user_ratings.sort_values(by='rating', ascending=False).merge(movies, on='movie_id',how='inner', suffixes=['_u', '_m']).head(20)

In [ ]:
recommendations = ratings[ratings['movie_id'].isin(user_ratings['movie_id']) == False][['movie_id']].drop_duplicates()
recommendations['prediction'] = recommendations.apply(lambda x: predict_rating(TEST_USER, x['movie_id']), axis=1)
recommendations.sort_values(by='prediction',
                          ascending=False).merge(movies,
                                                 on='movie_id',
                                                 how='inner',
                                                 suffixes=['_u', '_m']).head(20)